# Import Required Libraries

In [ ]:
#!pip install -q scikeras[tensorflow] albumentations

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import PIL
from PIL import Image
import albumentations as A
from collections import Counter

sns.set_style("whitegrid")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: "%.4f" % x)

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

#import lime
#from lime import lime_image
#from lime import submodular_pick
#from skimage.segmentation import mark_boundaries

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input as resnet101_preprocess
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenetv2_preprocess
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.densenet import preprocess_input as densenet201_preprocess
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as inceptionresnetv2_preprocess

from tensorflow.keras.preprocessing.image import load_img, img_to_array

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [ ]:
root_dir = "/mnt/d/Datasets/pc_parts/"

In [ ]:
folders = os.listdir(root_dir)
folders

In [ ]:
def read_data(folder):
    data, labels, paths = [], [], []
    for label in folders:
        path = f"{folder}/{label}/"
        folder_data = os.listdir(path)
        for image_path in folder_data:
            img = cv2.imread(path + image_path)
            img = cv2.resize(img, (224, 224))
            data.append(np.array(img))
            labels.append(label)
            paths.append(os.path.join(root_dir, label, image_path))

    return data, labels, paths

In [ ]:
all_data, all_labels, all_paths = read_data(root_dir)

In [ ]:
df = pd.DataFrame({
    'data': all_data,
    'filepath': all_paths,
    'labels': all_labels
})

In [ ]:
df.shape

In [ ]:
df_train, df_valid, df_test = np.split(df.sample(frac=1), [int(.6 * len(df)), int(.8 * len(df))])

print("Train Size:", df_train.shape[0])
print("Validation Size:", df_valid.shape[0])
print("Test Size:", df_test.shape[0])

# Visualization

In [ ]:
def plot_dist(df, target, title):
    plt.figure(figsize=(10, 8))
    ax = sns.countplot(y=target, order=df[target].value_counts().index, data=df, palette="crest")
    ax.set_title(title)
    for container in ax.containers:
        ax.bar_label(container)

In [ ]:
plot_dist(df_train, "labels", "Train - Categories")

In [ ]:
plot_dist(df_valid, "labels", "Validation - Categories")

In [ ]:
plot_dist(df_test, "labels", "Test - Categories")

In [ ]:
def visualize_df(df, data_col, target):
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))

    for i, ax in enumerate(axes.ravel()):
        if i < len(df):
            a = np.random.randint(1, len(df), 1)[0]
            img_path = df.loc[a][[data_col]].values[0]
            img_label = df.loc[a][[target]].values[0]

            image = Image.open(img_path).convert('RGB')

            ax.imshow(image)
            ax.set_title(img_label)
            ax.axis('off')
        
        else:
            ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
visualize_df(df, 'filepath', 'labels')

# Error Level Analysis

In [ ]:
def ela(image_path, temp_path, qualities):
    ela_images = []
    scale = 15
    original_img = cv2.imread(image_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

    for quality in qualities:
        cv2.imwrite(temp_path, original_img, [cv2.IMWRITE_JPEG_QUALITY, quality])

        temp_img = cv2.imread(temp_path)

        ela_img = scale * cv2.absdiff(original_img, temp_img)
        ela_images.append(ela_img)

    return ela_images

In [ ]:
image_path = df.sample()[['filepath']].values[0][0]
temp_path = "temp.jpg"
qualities = list(range(100, 0, -10))

ela_images = ela(image_path, temp_path, qualities)

fig, axes = plt.subplots(1, len(ela_images), figsize=(15, 5))
for i, ela_img in enumerate(ela_images):
    axes[i].imshow(cv2.cvtColor(ela_img, cv2.COLOR_BGR2RGB))
    axes[i].set_title(f"Quality: {qualities[i]}")
    axes[i].axis("off")

plt.tight_layout()
plt.show()

# Augmentation

In [ ]:
min_class_count = min(df_train['labels'].value_counts().values)
min_class_count

In [ ]:
transform = A.Compose([
    A.Resize(width=224, height=224),
    A.Rotate(limit=40),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.RandomGamma(p=0.2),
])

In [ ]:
def apply_augmentation(image_path, label):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    augmented_image = transform(image=image)['image']
    return augmented_image, label

In [ ]:
augmented_data = []
for index, row in df_train.iterrows():
    image_path = row['filepath']
    label = row['labels']
    augmented_image, augmented_label = apply_augmentation(image_path, label)
    augmented_data.append((augmented_image, image_path, augmented_label))

In [ ]:
augmented_df = pd.DataFrame(augmented_data, columns=['data', 'filepath', 'labels'])

In [ ]:
augmented_df.shape

In [ ]:
df_train.shape

In [ ]:
def visualize_aug_df(df, data_col, target):
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))

    for i, ax in enumerate(axes.ravel()):
        if i < len(df):
            a = np.random.randint(1, len(df), 1)[0]
            img_path = df.loc[a][[data_col]].values[0]
            img_label = df.loc[a][[target]].values[0]

            ax.imshow(img_path)
            ax.set_title(img_label)
            ax.axis('off')
        
        else:
            ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
visualize_aug_df(augmented_df, 'data', 'labels')

In [ ]:
augmented_df['labels'].value_counts()

In [ ]:
class_counts = Counter(df_train['labels'])
min_class_count = min(class_counts.values())

balanced_data = []
for label, count in class_counts.items():
    class_samples = df_train[df_train['labels'] == label].sample(min_class_count)
    balanced_data.append(class_samples)

balanced_df = pd.concat(balanced_data)

In [ ]:
balanced_df.shape

# Preprocess

In [ ]:
merged_df = pd.concat([df_train, augmented_df], ignore_index=True)

In [ ]:
class_counts = Counter(merged_df['labels'])
min_class_count = min(class_counts.values())

balanced_data = []
for label, count in class_counts.items():
    class_samples = merged_df[merged_df['labels'] == label].sample(min_class_count)
    balanced_data.append(class_samples)

balanced_df = pd.concat(balanced_data)

In [ ]:
balanced_df['labels'].value_counts()

In [ ]:
train_data =  np.array([np.array(val) for val in balanced_df['data'].values])
valid_data = np.array([np.array(val) for val in df_valid['data'].values])
test_data = np.array([np.array(val) for val in df_test['data'].values])

In [ ]:
le = LabelEncoder()
train_labels = le.fit_transform(balanced_df['labels'])
valid_labels = le.transform(df_valid['labels'])
test_labels = le.transform(df_test['labels'])

# Models

In [ ]:
scores_df = pd.DataFrame(columns=['Model Name', 'Batch Size', 'Learning Rate', 'Loss', 'Accuracy'])

In [ ]:
num_classes = len(folders)
print(num_classes)

In [ ]:
input_shape = (224, 224, 3)

In [ ]:
def train_tl_model(models, preprocessors, learning_rate=0.001, batch_size=64):
    for model_name, preprocess_func in zip(models, preprocessors):
        print()
        print(f' {model_name.__name__} '.center(50, '#'))
        input_layer = tf.keras.Input(input_shape)
        preprocessed_inputs = tf.keras.layers.Lambda(lambda x: preprocess_func(x))(input_layer)
        base_model = model_name(weights='imagenet', include_top=False, input_tensor=preprocessed_inputs)
        for layer in base_model.layers:
            layer.trainable = False
        inputs = base_model.input
        x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
        outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
        model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
        model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
        
        if not os.path.exists('./models'):
            os.mkdir('./models')
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=3)
        checkpoint = ModelCheckpoint(filepath=f'./models/{model_name.__name__}.h5', monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
        history = model.fit(train_data, train_labels, validation_data=(valid_data, valid_labels), epochs=50, batch_size=batch_size, callbacks=[early_stopping,checkpoint], verbose=0)

        fig, axes = plt.subplots(1, 2, figsize=(12, 3))
        ax = axes.ravel()

        for i, metric in enumerate(['accuracy', 'loss']):
            ax[i].plot(history.history[metric])
            ax[i].plot(history.history['val_' + metric])
            ax[i].set_xlabel('Epoch')
            ax[i].set_ylabel(metric)
            ax[i].legend(['train', 'valid'])

        plt.show()

        results = model.evaluate(test_data, test_labels, verbose=0)
        print(f'Test Loss: {results[0]}')
        print(f'Test Accuracy: {results[1]}')

        scores_df.loc[len(scores_df)] = [str(model_name.__name__), batch_size, learning_rate, results[0], results[1]]

        print(' '.center(50, '#'))
        print()

        if len(models) == 1:
            return model

In [ ]:
models = [
    #VGG16, 
    #VGG19, 
    ResNet50, 
    #ResNet101, 
    #MobileNetV2, 
    #DenseNet201, 
    #Xception, 
    #InceptionResNetV2
]

preprocessors = [
    #vgg16_preprocess, 
    #vgg19_preprocess, 
    resnet50_preprocess, 
    #resnet101_preprocess, 
    #mobilenetv2_preprocess,
    #densenet201_preprocess, 
    #xception_preprocess, 
    #inceptionresnetv2_preprocess
]

In [ ]:
model = train_tl_model(models, preprocessors)

# Hyperparameter Tuning

In [ ]:
def create_model(lr_rate=0.001):
    input_layer = tf.keras.Input(input_shape)
    preprocessed_inputs = tf.keras.layers.Lambda(lambda x: mobilenet_preprocess(x))(input_layer)
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=preprocessed_inputs)
    for layer in base_model.layers:
        layer.trainable = False
    inputs = base_model.input
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

In [ ]:
lr_rate = [0.1, 0.001]
batch_size = [128, 64]

In [ ]:
keras_model = KerasClassifier(
    build_fn=create_model, 
    lr_rate=lr_rate, 
    epochs=25, 
    batch_size=batch_size, 
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=3),
        ModelCheckpoint(filepath=f'./models/mobilnet.h5', monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
    ], 
    verbose=0
)

In [ ]:
param_grid = dict(lr_rate = lr_rate, batch_size = batch_size)

In [ ]:
#grid = GridSearchCV(estimator = keras_model, param_grid = param_grid, cv = None)
#grid_result = grid.fit(train_data, train_labels, validation_data=(valid_data, valid_labels))

In [ ]:
#print("The best parameters are:", grid_result.best_params_)

In [ ]:
#model = train_tl_model(models, preprocessors, learning_rate=0.001, batch_size=64)

In [ ]:
scores_df

# Test

In [ ]:
test_keyboard = root_dir + "keyboard/" + "1.jpg"
test_gpu = root_dir + "gpu/" + "1.jpg"

In [ ]:
def get_model_prediction(image_path):
    img = load_img(image_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = resnet50_preprocess(x)
    predictions = model.predict(x, verbose=0)
    return le.classes_[predictions.argmax()]

In [ ]:
get_model_prediction(test_keyboard)

In [ ]:
get_model_prediction(test_gpu)

In [ ]:
pred = []
for file in df_test['filepath'].values:
    pred.append(get_model_prediction(file))

In [ ]:
pred = np.array(pred)

# Model Explainability

In [ ]:
# https://keras.io/examples/vision/grad_cam/
def get_img_array(img_path, size):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    array = tf.keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size "size"
    array = np.expand_dims(array, axis=0)
    return array

# https://keras.io/examples/vision/grad_cam/
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# https://keras.io/examples/vision/grad_cam/
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)
    # Save the superimposed image
    superimposed_img.save(cam_path)

    return cam_path

In [ ]:
last_conv_layer_name = "conv5_block3_3_conv"


In [ ]:
model.layers[-1].activation = None

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(15, 10))
random_index = np.random.randint(0, len(df_test), 16)

for i, ax in enumerate(axes.ravel()):
    img_path = df_test['filepath'].iloc[random_index[i]]
    img_array = resnet50_preprocess(get_img_array(img_path, size=input_shape))
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    cam_path = save_and_display_gradcam(img_path, heatmap)
    ax.imshow(plt.imread(cam_path))
    ax.axis('off')
    
    if df_test['labels'].iloc[random_index[i]] == pred[random_index[i]]:
        color = "green"
    else:
        color = "red"
    ax.set_title(f"True: {df_test['labels'].iloc[random_index[i]]}\nPredicted: {pred[random_index[i]]}", color=color)
plt.tight_layout()
plt.show()

# Results

In [ ]:
def plot_scores(df):
    plt.figure(figsize=(15, 20))

    plt.subplot(211)
    ax = sns.barplot(data=df, x='Model Name', y="Accuracy", palette='viridis')
    for container in ax.containers:
        ax.bar_label(container)

    plt.xlabel('')
    plt.xticks(rotation=45)
    plt.title("Model / Test Accuracy")

    plt.subplot(212)
    ax = sns.barplot(data=df, x='Model Name', y="Loss", palette='viridis')
    for container in ax.containers:
        ax.bar_label(container)

    plt.xlabel('')
    plt.xticks(rotation=45)
    plt.title("Model / Test Loss")

    plt.show()

In [ ]:
plot_scores(scores_df)